In [1]:
import numpy as np
import pandas as pd

In [2]:
#Loading all datasets into memory
items = pd.read_csv('../data/raw/olist_order_items_dataset.csv', encoding='utf-8')
sellers = pd.read_csv('../data/raw/olist_sellers_dataset.csv', encoding='utf-8')
leads = pd.read_csv('../data/raw/olist_marketing_qualified_leads_dataset.csv', encoding='utf-8')
deals = pd.read_csv('../data/raw/olist_closed_deals_dataset.csv', encoding='utf-8')
orders = pd.read_csv('../data/raw/olist_orders_dataset.csv', encoding='utf-8')

In [3]:
#Dropping unnecessary variables of dataframes
items = items.drop(['shipping_limit_date'], axis=1)
sellers = sellers.drop(['seller_city', 'seller_zip_code_prefix'], axis=1)
deals = deals.drop(['sdr_id','sr_id'], axis=1)

In [4]:
#Merging deals and leads to marketing dataset
marketing = leads.merge(deals, on='mql_id', how='left')
marketing = marketing.dropna(subset=['seller_id']).set_index('mql_id')

In [5]:
#Transform "declared"-Variables into boolean variables
marketing.declared_monthly_revenue = marketing.declared_monthly_revenue.replace(to_replace=0.0 ,value= np.nan)
marketing['declare_revenue'] = ['False' if pd.isna(x) == True else 'True' for x in marketing.declared_monthly_revenue]
marketing['declare_opc'] = ['False' if pd.isna(x) == True else 'True' for x in marketing.declared_product_catalog_size]
marketing = marketing.drop(['declared_monthly_revenue','declared_product_catalog_size'], axis=1)

In [6]:
#Transform average_stock into boolean variable due to only 62 filled cases
marketing.average_stock = marketing.average_stock.replace(to_replace= ['unknown'] ,value= np.nan)
marketing['has_stock'] = ['False' if pd.isna(x) == True else 'True' for x in marketing.average_stock]
marketing = marketing.drop('average_stock', axis=1)

In [7]:
#Complete 'has'-Variables with fill out NaNs into false statements
marketing.has_company = marketing.has_company.replace(to_replace= [np.nan] ,value= 'false')
marketing.has_gtin = marketing.has_gtin.replace(to_replace= [np.nan] ,value= 'false')

In [8]:
#Transform contact_date and won_date into calculated days2conversion
marketing.first_contact_date = pd.to_datetime(marketing.first_contact_date).dt.date.astype('datetime64')
marketing.won_date = pd.to_datetime(marketing.won_date).dt.date.astype('datetime64')
marketing['days2conversion'] = (marketing.won_date - marketing.first_contact_date).dt.days
marketing = marketing.drop(['won_date','first_contact_date'], axis=1)

In [9]:
#Convert "unknown" category in origin to NANs
marketing.origin = marketing.origin.replace(to_replace=['unknown'], value=np.nan)

In [10]:
#Merging orders with sellers and items dataset to commerce
commerce = pd.merge(orders, items, on='order_id')
commerce = commerce.merge(sellers, on='seller_id')

In [11]:
#Calculation of sales volume per unique seller id
orders_revenues = commerce[['order_id','price']].groupby(by=commerce.order_id).sum()
commerce = commerce.merge(orders_revenues,on='order_id')
seller_revenues = commerce[['seller_id', 'price_y']].groupby(commerce.seller_id).sum().round(2)
seller_revenues =  dict(zip(seller_revenues.index,seller_revenues.price_y))

In [12]:
#Calculation of customer orders per unique seller id
unique_sellers = commerce.seller_id.unique()
counts = []

for seller in unique_sellers:
    counter = 0

    for row in commerce.seller_id:
        if row == seller:
            counter += 1
        else:
            counter = counter
    
    counts.append(counter)

seller_orders = dict(zip(unique_sellers,counts))

In [13]:
# Calculation of last order activity per unique seller id
latest = max(pd.to_datetime(commerce.order_purchase_timestamp).dt.date)
start = min(pd.to_datetime(commerce.order_purchase_timestamp).dt.date)
timestamps = {}

for seller, time in zip(commerce.seller_id, pd.to_datetime(commerce.order_purchase_timestamp).dt.date):
    if seller in timestamps:
        if time > timestamps[seller]:
            timestamps[seller] = time
        else:
            timestamps[seller] = timestamps[seller]
    else:
        timestamps[seller] = start

for key in timestamps:
    timestamps[key] = (latest - timestamps[key]).days

In [14]:
scores = pd.DataFrame()
scores['seller_id'] = unique_sellers
scores['revenues'] = scores.seller_id.map(seller_revenues)
scores['count_orders'] = scores.seller_id.map(seller_orders)
scores['days_last_activity'] = scores.seller_id.map(timestamps)

In [15]:
labels = range(1,6)
scores['recency'] = pd.qcut(scores.days_last_activity,q=5, labels=labels)
scores['frequency'] = pd.qcut(scores.count_orders,q=5, labels=labels)
scores['monetary_ratio'] = pd.qcut(scores.revenues,q=5, labels=labels)
scores['rfm_score'] = scores[['recency','frequency', 'monetary_ratio']].mean(axis=1).round(0)

In [16]:
final = marketing.merge(scores[['seller_id','rfm_score']], on='seller_id', how='left')
final = final.dropna(subset=['rfm_score'])